In [1]:
import pandas as pd
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import shift
from sklearn.metrics import r2_score as reg_acc
from sklearn.neural_network import MLPRegressor as NN
from statsmodels.tsa.arima_model import ARIMA
from sklearn.neighbors import KNeighborsRegressor as KNN

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
features=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/features.csv')
print(features.shape)
features.head()

(8190, 12)


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
stores=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/stores.csv')
print(stores.shape)
stores.head()

(45, 3)


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [5]:
train=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/train.csv')
print(train.shape)
train.head()

(421570, 5)


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [6]:
merged_1=train.merge(stores, on=['Store'], how='left')
print(merged_1.shape)
merged_1.head()

(421570, 7)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315
3,1,1,2010-02-26,19403.54,False,A,151315
4,1,1,2010-03-05,21827.90,False,A,151315


In [7]:
sum(train['Store']==1)

10244

In [8]:
train=merged_1.merge(features, on=['Store','Date','IsHoliday'], how='left')
print(train.shape)
train.head()

(421570, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


In [0]:
nan_values=train.isnull()
nan_values_count=[0]*train.shape[1]
for i in range(train.shape[1]):
  nan_values_count[i]=sum(nan_values.iloc[:,i])

In [10]:
columns=list(train.columns.values)
ratios=np.array(nan_values_count)/train.shape[0]
print('Nan values count in each column and their ratio with total data size')
for i in range(len(columns)):
  print(columns[i],nan_values_count[i],ratios[i])


Nan values count in each column and their ratio with total data size
Store 0 0.0
Dept 0 0.0
Date 0 0.0
Weekly_Sales 0 0.0
IsHoliday 0 0.0
Type 0 0.0
Size 0 0.0
Temperature 0 0.0
Fuel_Price 0 0.0
MarkDown1 270889 0.6425718148824632
MarkDown2 310322 0.7361102545247528
MarkDown3 284479 0.6748084541120098
MarkDown4 286603 0.6798467632896079
MarkDown5 270138 0.6407903788220224
CPI 0 0.0
Unemployment 0 0.0


## **Only markdown columns contain the nan values. Since these values are continuous we can just replace unknown values by zero**

In [11]:
train['MarkDown1']=train['MarkDown1'].fillna(0)
train['MarkDown2']=train['MarkDown2'].fillna(0)
train['MarkDown3']=train['MarkDown3'].fillna(0)
train['MarkDown4']=train['MarkDown4'].fillna(0)
train['MarkDown5']=train['MarkDown5'].fillna(0)
print('missing values removed')

missing values removed


In [12]:
nan_values=train.isnull()
nan_values_count=[0]*train.shape[1]
for i in range(train.shape[1]):
  nan_values_count[i]=sum(nan_values.iloc[:,i])
columns=list(train.columns.values)
ratios=np.array(nan_values_count)/train.shape[0]
print('Nan values count in each column and their ratio with total data size')
for i in range(len(columns)):
  print(columns[i],nan_values_count[i],ratios[i])

Nan values count in each column and their ratio with total data size
Store 0 0.0
Dept 0 0.0
Date 0 0.0
Weekly_Sales 0 0.0
IsHoliday 0 0.0
Type 0 0.0
Size 0 0.0
Temperature 0 0.0
Fuel_Price 0 0.0
MarkDown1 0 0.0
MarkDown2 0 0.0
MarkDown3 0 0.0
MarkDown4 0 0.0
MarkDown5 0 0.0
CPI 0 0.0
Unemployment 0 0.0


In [0]:
train['month']=pd.DatetimeIndex(train['Date']).month

In [0]:
train=train.drop('Date',axis=1)

In [15]:
test=pd.read_csv('gdrive/My Drive/DS Project/Dataset/Dataset 2/test.csv')
print(test.shape)
test.head()
print(test.iloc[test.shape[0]-1,:])

(115064, 4)
Store                45
Dept                 98
Date         2013-07-26
IsHoliday         False
Name: 115063, dtype: object


In [16]:
merged_1=test.merge(stores, on=['Store'], how='left')
test=merged_1.merge(features, on=['Store','Date','IsHoliday'], how='left')
nan_values=test.isnull()
nan_values_count=[0]*test.shape[1]
for i in range(test.shape[1]):
  nan_values_count[i]=sum(nan_values.iloc[:,i])
test['MarkDown1']=test['MarkDown1'].fillna(0)
test['MarkDown2']=test['MarkDown2'].fillna(0)
test['MarkDown3']=test['MarkDown3'].fillna(0)
test['MarkDown4']=test['MarkDown4'].fillna(0)
test['MarkDown5']=test['MarkDown5'].fillna(0)
print('missing values removed')
test['month']=pd.DatetimeIndex(test['Date']).month
test=test.drop('Date',axis=1)

missing values removed


In [0]:
train['sales']=train['Weekly_Sales']
train=train.drop('Weekly_Sales',axis=1)

In [18]:
print(train.shape)
print(test.shape)

(421570, 16)
(115064, 15)


Training data is from date 2010-02-05 to 2012-10-26 

test data is from date 2012-11-02 to 2013-07-26

Let's construct a very simple model that only takes past sale values and predict future values without using the features

In [19]:
store_1_sales=np.array(data['Weekly_Sales'][data['Store']==1])

NameError: ignored

In [0]:
plt.plot(store_1_sales)
plt.title('Sales of Store 1')

Let's contruct the training data to predict sales. We can do this by using 100 days sales as input and output will be 101th day sale. Similarly, the next data point will be sales from day 2 to day 101 and the output will be sales of day 102.

In [0]:
features=1000
count=store_1_sales.shape[0]-features-1
X=np.zeros((count,features))
Y=np.zeros(count)
for i in range(count):
  X[i,:]=store_1_sales[i:features+i]
  Y[i]=store_1_sales[features+i+1]

In [0]:
scale=np.max(X)
X=X/scale
Y=Y/scale

In [0]:
#X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
train_size=int(X.shape[0]*0.8)
X_train=X[:train_size]
y_train=Y[:train_size]
X_test=X[train_size:]
y_test=Y[train_size:]

In [0]:
reg = LinearRegression().fit(X_train, y_train)

In [0]:
prev=X_test[0,:]
prediction=np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
  out=reg.predict(prev.reshape(1,1000))
  prev=shift(prev, -1, cval=out)
  prediction[i]=out

In [0]:
print('Test accuracy: ',reg_acc(y_test,prediction))

In [0]:
plt.figure()
plt.plot(y_test)
plt.title('Actual sales')
plt.show()
plt.figure()
plt.plot(prediction)
plt.title('Predicted sales')
plt.show()

In [0]:
reg = NN(hidden_layer_sizes=(100,100,100)).fit(X_train, y_train)

In [0]:
prev=X_test[0,:]
prediction=np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
  out=reg.predict(prev.reshape(1,1000))
  prev=shift(prev, -1, cval=out)
  prediction[i]=out
print('Test accuracy: ',reg_acc(y_test,prediction))
plt.figure()
plt.plot(y_test*scale)
plt.title('Actual sales')
plt.show()
plt.figure()
plt.plot(prediction*scale)
plt.title('Predicted sales')
plt.show()

In [0]:
reg = KNN().fit(X_train, y_train)
prev=X_test[0,:]
prediction=np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
  out=reg.predict(prev.reshape(1,1000))
  prev=shift(prev, -1, cval=out)
  prediction[i]=out
print('Test accuracy: ',reg_acc(y_test,prediction))
plt.figure()
plt.plot(y_test*scale)
plt.title('Actual sales')
plt.show()
plt.figure()
plt.plot(prediction*scale)
plt.title('Predicted sales')
plt.show()

In [0]:
'''
p=5
d=1
q=0
train=store_1_sales[:int(store_1_sales.shape[0]*0.8)]
test=store_1_sales[int(store_1_sales.shape[0]*0.8):]
train=train/scale
test=test/scale
prev = [x for x in train]
prediction =[]
for t in range(len(test)):
  model = ARIMA(prev,order=(p,d,q))  #, order=(p,d,q))
  model_fit = model.fit(disp=0)
  output = model_fit.forecast()
  yhat = output[0]
  #print(yhat)
  prediction.append(yhat)
  obs = yhat
  prev.append(obs)
  '''

In [0]:
indices=np.arange(test.shape[0])
plt.figure()
plt.plot(indices,test,'b',indices,prediction,'r')
plt.title('Actual vs Predicted sales')
plt.show()

In [0]:
'''
indices=np.arange(len(test))
plt.plot(indices,test,'b',indices,prediction,'r')
plt.title('Actual sales VS Predicted sales')
plt.show()
'''

In [0]:
#reg_acc(test,prediction)

**Deep learning**

In [0]:
#!pip3 install torch torchvision
#!pip3 install torch --upgrade

In [0]:
import torch
from torch import nn

**Multivariable Deep learning**

In [0]:
import keras

In [0]:
data